# run_model_v2
This is a script used to classify images, using Tensorflow 'floor' models exported from Lobe.ai. See learning brief for more details about the project and what is expected to be able to run this script effectively. 

## File and Folder Structure

Create a permanent folder on your hard drive which will serve as your base folder for the model, add the following subfolders and scripts to this base folder:
Subfolders:
1.	csv - to store the CSV exports from Ona
2.	images – subfolders for each region can be created under this folder; this is where the model will store downloaded images
3.	models – stores the Lobe models
4.	outputs – model outputs are placed here (CSV format)
5.	src – extra code files
Scripts:
1.	image_downloader.py – this script will download batches of images from Ona, and shrink/resize them for the model
2.	run_model.py – this script runs on a folder of images for the selected model (door, floor, structure), leverages the TensorFlow model, and outputs classifications

## Import Statements

The following block contains import statements. Required packages include pandas, os, PIL, sys, and time. Note that 'src' is the source folder set up in the same directory, NOT a downloaded package. This will confirm that folders are set up correctly as well. User must set line 5 in ths block their own base folder for this project.

In [1]:
import pandas as pd
import os
from PIL import Image
import sys
sys.path.insert(0, "C:\\Users\\lauren.allognon\\USHA-ML") #update pathname
from src.tf_loader2 import TFModel
import time

ModuleNotFoundError: No module named 'src'

In [ ]:
#set current directory as root
ROOT = os.path.dirname(__file__)

In [ ]:
# run this
if __name__ == '__main__':
    #get inputs from user
    image_dir = input("What is the subfolder in images? ")
    
    #Check if the user has original CSV to merge results with
    csv_check = input("Do you have a CSV with image URLS to merge the results with? Y/N")
    if(csv_check=='Y'): 
        csv_file = input("What is the name of the CSV file? ")
        col = input('What is the column name of image URLs? ')
    
    model_name = input("What model are you evaluating? (1) Superstructure (2) Interface (3) Presence of door ")

    #load selected model
    if int(model_name) == 1:
        model = TFModel(model_dir=os.path.join(ROOT,'models','structure'))
    elif int(model_name) == 2:
        model = TFModel(model_dir=os.path.join(ROOT,'models','slab'))
    elif int(model_name) == 3:
        model = TFModel(model_dir=os.path.join(ROOT,'models','door'))
    else:
        print('Invalid model')
        sys.exit()
    model.load()

    images = os.listdir(os.path.join(ROOT,'images',image_dir))
    results = pd.DataFrame()
    #run model and append results to dataframe
    for image in images:
        output = run_model(os.path.join(ROOT,'images',image_dir, image))
        result = pd.DataFrame(output['predictions']).set_index('label').T
        result['image'] = image
        results = results.append(result)
    results.reset_index(drop=True, inplace=True)
    
    cols = results.columns[:-1]
    results['max_conf'] = results[cols].max(axis=1)
    results['doubt'] = (results.loc[:, cols] > .25).sum(axis=1)
    results['accuracy'] = pd.cut(results.max_conf, bins=[0,.5,.85,.95,1], labels=['V Low', 'Low', 'Med', 'High'])
    results['model_output'] = results[cols].idxmax(axis=1)

    #If user gives csv, merge with results, else return a csv of just the results
    if(csv_check =='Y'):
        data = pd.read_csv(os.path.join(ROOT,'csv',csv_file),index_col='uuid')
        data['image'] = data[col].str[-17:]
        data = data.merge(results, how='left', on='image')
    else:
        data = results
    
    #output to CSV
    fname = 'output-' + time.strftime('%d%m%Y-%H%M') + '.csv'
    data.to_csv(os.path.join(ROOT,'outputs',fname))
    print('Saved ' + fname)

## run_model Function
Definition of run_model function, used to open the image and run the model. It returns a dictionary with 'predictions' as the key and the value is a list containing a dictionary per output label containing keys 'label' and 'confidence'. CLEAN THIS DESCRIPTION UP

In [ ]:
#function to open image and run model
#returns nested objects:
#single item dict w key 'predictions' and value:
#   list containing one dict per output label containing keys:
#       label, confidence
def run_model(image):
    if os.path.isfile(image):
        image = Image.open(image)
        outputs = model.predict(image)
        print(f"Predicted: {outputs}")
        return outputs
    else:
        print(f"Couldn't find image file {image}")

## Main Script
First it gets input from the user, asking for the subfolder where the images are stored (if used in tandem with the image downloader, this is where those images were downloaded to). Then, it asks if you have a CSV file (like the kind used for the image_downloader script), with the column name for the column containing image URLS, to which the results of the model will be appended. If not, the CSV that will be created will just contain the results of running the model (the image, the max confidence, the doubt, the accuracy, and the model output). Finally, the user is prompted to type 1, 2 or 3, to choose which model. 1 corresponds to the 'superstructure' model, 2 to 'Interfac'e, and 3 to 'Presence of door'. More information can befound in the learning brief for this project. 